# FAIR and scalable management of small-angle X-ray scattering data  
## Module 3: Data analysis and visualisation
### 3.2: Analysis

> Authors: Torsten Giess, Selina Itzigehl, Jan Range, Johanna Bruckner, Juergen Pleiss  
> Last modified: 02.05.2022

---

### **Abstract** <a class="anchor" name="abstract"></a>

Lorem ipsum dolor ...

---

### **Table of Contents** <a class="anchor" name="table_of_contents"></a>

- [Abstract](#abstract)
- [Workflow](#workflow)
    - [User guide](#user_guide)
    - [Preparation](#preparation)
    - [Analysis with Python](#analysis)
- [Disclosure](#disclosure)

---

### **Workflow** <a class="anchor" name="workflow"></a>

Following is the workflow for Module 3.2: Analysis of FAIR and scalable management of small-angle X-ray scattering data.

#### **User guide** <a class="anchor" name="user_guide"></a>

Lorem ipsum...

#### **Preparation** <a class="anchor" name="preparation"></a>

This section contains the necessary preparations for using this module. Code cells in this section are required regardless of which functionality of this notebook is used. First, the required packages from the [Python 3 standard library](https://docs.python.org/3/library/), the Python Package Index ([PyPI](https://pypi.org/)), and *ad hoc* modules of this work are imported. Then, both current time and path are retrieved and stored in the desired formats.

In [ ]:
print("Importing standard library packages.")
from datetime import date
from pathlib import Path
from typing import List
print ("Done.")

In [ ]:
print("Importing PyPI packages.")
import numpy as np
import pandas as pd
from pyaniml import AnIMLDocument
print ("Done.")

In [ ]:
print("Importing local packages.")
from modules.originreader import LorentzianReader
print("All done.")

In [ ]:
date_suffix = str(date.today()).replace("-", "")[2:]

In [ ]:
cwd = Path.cwd()
path_to_datasets = cwd / "./datasets/"

In [ ]:
def calculate_scattering_vector(d: float) -> float:
    q = (2 * np.pi) / (d / 10)
    return q

def calculate_linear_regression(m: float, x: float, b: float) -> float:
    y = m * x + b
    return y

def calculate_lattice_plane(q: float) -> float:
    d = (2 * np.pi) / q
    return d

def calculate_lattice_ratio(d: float, d_0: float) -> float:
    d_ratio = d / d_0
    return d_ratio

def determine_phase(d_ratios: List) -> bool:
    H1 = [
        (1 / np.sqrt(3)),
        (1 / np.sqrt(4)),
        (1 / np.sqrt(7)),
        (1 / np.sqrt(9)),
    ]
    V1 = [
        (1 / np.sqrt(2)),
        (1 / np.sqrt(3)),
        (1 / np.sqrt(4)),
        (1 / np.sqrt(5)),
    ]
    La = [(1 / 2), (1 / 3), (1 / 4), (1 / 5)]
    
    for i, j in enumerate(d_ratios):
        if (abs(d_ratios[i] - H1[i])) < 0.03:
            return "hexagonal"
        elif (abs(d_ratios[i] - V1[i])) < 0.03:
            return "cubic"
        elif (abs(d_ratios[i] - La[i])) < 0.03:
            return "lamellar"
        else:
            return "indeterminate"

def calculate_a_H1(d: float, h: int, k: int) -> float:
    a_H1 = d * np.sqrt((4/3)*((h**2 + k**2 + (h * k))))
    return a_H1


def calculate_a_V1(d: float, h: int, k: int, l: int) -> float:
    a_V1 = d * (np.sqrt((h ** 2) + (k ** 2) + (l ** 2)))
    return a_V1

---

#### **Analysis with Python** <a class="anchor" name="analysis"></a>

lorem ipsum...

Import Lorentzian data from Origin TXT output file:

In [ ]:
available_txt_files = [file for file in (path_to_datasets / "raw/test/").glob("*.txt")]
print([file.name for file in available_txt_files])

In [ ]:
dict_of_df = {}
i=0
for file in available_txt_files:
    dict_of_df[available_txt_files[i].name] = LorentzianReader(available_txt_files[i]).get_xc_dataframe()
    i += 1
print(dict_of_df.keys())

In [ ]:
d_from_literature = [52.49824535, 26.24912267, 17.49941512]
q_cholpal_literature = [calculate_scattering_vector(d) for d in d_from_literature]

In [ ]:
slope, intercept = np.polyfit(
    x=dict_of_df[available_txt_files[0].name]["value"].tolist(),
    y=q_cholpal_literature,
    deg=1
)
q_corrected = [calculate_linear_regression(slope, value, intercept) for value in dict_of_df[available_txt_files[2].name]["value"].tolist()]
print(q_corrected)

In [ ]:
d_measured = [calculate_lattice_plane(q) for q in q_corrected]
print(d_measured)

In [ ]:
d_ratio = [calculate_lattice_ratio(d, d_measured[0]) for d in d_measured[1:]]
print(d_ratio)

In [ ]:
phase = determine_phase(d_ratio)
if phase == "hexagonal":
    h = [1, 1, 2, 2, 3]
    k = [0, 1, 0, 1, 0]
    a_hex = []
    for i, j in enumerate(d_measured):
        a_i = calculate_a_H1(d_measured[i], h[i], k[i])
        a_hex.append(a_i)
    phase_information = [phase, (np.mean(a_hex))]

elif phase == "cubic":
    h = [1, 1, 2, 2, 2]
    k = [0, 1, 0, 1, 2]
    l = [0, 1, 0, 1, 2]
    a_cub = []
    for i, j in enumerate(d_measured):
        a_i = calculate_a_V1(d_measured[i], h[i], k[i], l[i])
        a_cub.append(a_i)
    phase_information = [phase, (np.mean(a_cub))]

elif phase == "lamellar":
    phase_information = [phase, d_measured[0]]

else:
    phase_information = ["indeterminate", "-"]

In [ ]:
print(d_measured)

In [ ]:
print(phase_information)

---

### **Disclosure** <a class="anchor" name="disclosure"></a>

**Contributions**

If you wish to contribute to the FAIR Chemistry project, find us on [GitHub](https://github.com/FAIRChemistry)!

**MIT License**

Copyright (c) 2022 FAIR Chemistry

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.